<a href="https://colab.research.google.com/github/disha-002/DeepLearning/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd

df=pd.read_csv("glass.csv")

#reading the data

In [53]:
df.shape
# finding the number of columns and rows

(214, 10)

In [54]:
df.columns
# figuring out the columns of our data

Index(['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type'], dtype='object')

In [55]:
df.head()
# looking at few of the records

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [56]:
df["y"]=(df["Type"]==1).astype(int)
df.drop(columns=["Type"])

# we are converting our problem into a binary problem , where we take type 1 galss a positive class and others as negative class

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,y
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,0
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,0
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,0
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,0


In [57]:
X=df.drop(columns=["y"]).values
y=df["y"].values

# separating the input and output columns

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,y,test_size=0.2,random_state=42
)

# splitting the data into training and testing because a model cannot be tested using the training data , it will give false confidence

In [59]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# since the range of all the features are different we need to scale it to so no feature dominates the other

In [60]:
import numpy as np

def sigmoid(z):
  return (1/(1+np.exp(-z)))

# to give confidence values instead of decision i.e 'yes' or 'no' we make use of sigmoid function that gives us probabilistic values

In [61]:
def predict_prob(X,w,b):
  z=X @ w +b
  p=sigmoid(z)

  return p
# depending on the evidence score we find the confidence value

In [62]:
def loss(y,p):
  return -np.mean(y*np.log(p) + (1-y)*np.log(1-p))

# since there is no prediction here , and we are involved with confidence value , we do not see the correctness of the prediction.
# rather we see the fault in the confidence value , giving more penalty to confident wrong prediction

In [63]:
def update_weights(X,y,w,b,lr):
  p=predict_prob(X,w,b)

  error=p-y
  #one question that i have here is p is the confidence value and the values of y is 0 and 1
  #so why are we subtracting them to get the error
  w=w-lr*(X.T @ error )/len(y)
  b=b-lr*np.mean(error)

  return w,b

#since we need to train the model to give confident correct prediction , we have to make sure it learns from its mistakes at a rate of lr

In [64]:
w=np.zeros(X_train.shape[1])
b=0.0
lr=0.1
epochs=100

for i in range(epochs):
  w,b=update_weights(X_train,y_train,w,b,lr)

# we train the model starting with weights and bias as 0 , as the model learns the weights and biases improves

In [65]:
def predict_label(p,threshold=0.5):
  return (p>=threshold).astype(int)

# we have set a threshold value and depending on that ,the confidence value is used to predict the type

In [66]:
p=predict_prob(X_test,w,b)
prediction=predict_label(p)
l=loss(y_test,p)

print(prediction,y_test)
print(l)

#I tested taking thresholds 0.5,0.7, 0.9 all of them gave loss approximately the same
# to answer why higher threshold is better in glass quality control, i think the reason could be because different quality of glasses are used in making different things for example , window panes , glass for drinking water. So, choosing a small threshold might classify them wrongly resulting in being used somewhere else.

[1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0
 1 0 0 0 0 1] [1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0
 0 0 0 0 0 1]
0.33135332129598355


**How it differs from perceptron ?**

This differs from perceptron because it uses step function and gives a decision , here instead of step function we are using sigmoid function that gives us the confidence of the value , how close it is to the correct value.

**Why sigmoid matters ?**

Sigmoid matters because it gives the confidence value of the prediction.Say, it gives 0.6 as the probability, so it means that it is 60% confident that the prediction is correct.

**What problem still remains unsolved?**

Since we converted the problem to a binary problem , giving type 1 glass a positive class while others a negative class , I think we have not solved the problem of predicting other types.